# Downloading CAPES data

This notebook shows how we used the [ckan API](https://docs.ckan.org/en/2.9/api/index.html#) to download open access data from [Dados Abertos CAPES](https://dadosabertos.capes.gov.br/) regarding the evaluation of Postgraduate Programs in Brazil.

In [33]:
import requests
import json
import os

In [34]:
# This url will be used to request a full json representation of each dataset from Dados Abertos CAPES
url = 'https://dadosabertos.capes.gov.br/api/3/action/package_search'

In [35]:
def get_api_response(url):
    response = requests.get(url)
    response_dict = json.loads(response.text)
    return response_dict

In [36]:
def extract_links(response_dict):
    results = response_dict['result']['results'] #Getting only the 'results' portion of the json file
    links = {}
    for result in results:
        name = result['name']
        links[name] = []
        for resource in result['resources']:
            links[name].append(resource['url'])
    return links

In [42]:
def create_dir(path):
    os.makedirs(path, exist_ok=True)

In [43]:
def download_files(links, outdir='.'):
    for k, v in links.items():
        path = f"{outdir}/{k}"
        create_dir(path)
        for url in v:
            filename = url.rsplit('/',1)[-1]
            if filename.endswith("csv"): #Only download .csv files
                print(f'Downloading {filename}')
                response = requests.get(url)
                open(f'{path}/{filename}', 'w').write(response.text) #Response.text is the content in unicode; response.content is the content in bytes

In [44]:
test = get_api_response(url)
links = extract_links(test)
download_files(links, outdir='data')